### MTH 610 Model Constrained Optimization 1 (Homework 2)

##### Emily Bogle
##### 10/17/23

#### Problem 

Consider the boundary value problem (BVP) for a function $x:[0,1] \to \mathbb{R}$,
$$
\begin{cases}
-x''(t)+x^3(t)=u\\
(x(0)=0, \quad x(1)=0
\end{cases}
$$
where $u\in\mathbb{R}$ is a constant scalar parameter. In this assignment we want to find an optimal parameter value $u^*$ such that the solution $x(t)$ to the BVP minimizes the cost functional 
$$
J(x,u)=\frac{\omega}{2}|u|^2+\frac{1}{2}\int_0^1|x(t)-\overline{x}(t)|^2dt
$$
where $\overline{x}(t)$ is a given function (desired outcome/goal, measurement) and $\omega\geq 0$ is a specified fixed weight parameter. 

#### Part 1 (Optimality Conditions)

We know that 
$$
\delta J = \omega u \delta u +\int_0^1 (x(t)-\overline{x}(t))\delta x dt.
$$
Now, the variation in the model is given by the following:


$$
\begin{cases}
-\delta x'' + 3x^2\delta x = \delta u \\
\delta x(0)=0, \quad \delta x(1)=0
\end{cases}
$$

We will take the inner product with the lambda function to get,
$$
\int_0^1-\delta x''\lambda + 3x^2\delta x \lambda dt = \int_0^1\delta u \lambda dt \\
\implies -\delta x'\lambda |_0^1 +\int_0^1 \delta x'\lambda' + 3x^2\delta x \lambda dt = \int_0^1 \delta u dt\\
\implies (-\delta x'\lambda + \delta x \lambda')|_0^1 + \int_0^1 -\lambda''\delta x + 3x^2\lambda \delta x dt = \int_0^1 \delta u \lambda dt \\
\implies -\delta x'(1) \lambda (1) + \delta x(1) \lambda' (1) - (-\delta x'(0) \lambda (0) + \delta x(0) \lambda' (0)) + \int_0^1 -\lambda'' \delta (x) + 3x^2 \lambda \delta (x) dt = \int_0^1 \delta u \lambda dt
$$

Therfore, we define $\lambda (t)$ solution to the BVP (adjoint model) as the following:
$$
\begin{cases}
-\lambda''(t) +3(x(t))^2 \lambda(t) = x(t) - \overline{x}(t) \\
\lambda (0) =0, \quad \lambda(1)=0
\end{cases}
$$

With this definition of $\lambda (t)$ we obtain $\delta J$ in terms of $\delta u$,
$$
\begin{align*}
\delta J &= \omega u \delta u +\int_0^1 \delta u \lambda(t) dt\\
&= [\omega u + \int_0^1 \lambda(t) dt]\delta u
\end{align*}
$$

Now, the optimality condition ($\nabla_u J=0$) is given by,
$$
\omega u + \int_0^1 \lambda(t) dt =0
$$

Therefore our first order optimality conditions (Euler Lagrange Equations) for the minmization of the cost functional subject the the BVP constraint are given by,
$$
\begin{cases}
-x''(t)+x^3(t)=u\\
-\lambda''(t) + 3(x(t))^2\lambda(t) = x(t)-\overline{x}(x)\\
x(0)=0, \quad x(1)=0\\
\lambda(0)=0, \quad \lambda(1)=0
\end{cases}
$$
with the optimality condition:
$$
\omega u +\int_0^1 \lambda(t)dt =0
$$
where we know $\nabla_u J = \omega u +\int_0^1 \lambda(t)dt$.

#### Part 2 (Use Method of Choice to Estimate $u^*$)

With the given optimality system we can discretize and then optimize. After discretizing the two systems of PDEs both become a system of nonlinear equations to solve. Furthermore, in this way we can start wtih an initial $u$ value, and then from that we can find $x$ givne the model BVP constraint, and then once we have $x$ we can solve for $\lambda$ using the adjoint model. Then we will cycle through this process in doing steepest descent iterations on $\nabla_u J$. 

A discrete version to the original BVP is obtained by considering a uniform partition of the interval $[0,1]$ with nodes $\{t_i:i=0:n+1\}$ at an increment $h=\frac{1}{N+1}$,
$$
0=t_0<t_1<\dots<t_N<t_{N+1}=1, \quad t_i=i*h \textrm{ for } i=0:N+1
$$
and an approximation for the second order derivative using a finite difference formula is 
$$
x''(t_i)\approx\frac{x(t_{i+1})-2x(t_i)+x(t_{i-1})}{h^2}, \quad i=1:N
$$

Therefore a discrete version of the BVP is given by,
$$
\begin{cases}
\frac{x_{i+1}-2x_i+x_{i-1}}{h^2} + x^3_{i} = u_i, \quad i=1:N\\
x_0=0, \quad x_{N+1}=0
\end{cases}
$$
where $x_i\approx x(t_i), \quad i=1:N$.

We can write this sytem of equations in matrix vector format,
$$
\mathbf{A}\mathbf{x}+\mathbf{G}(\mathbf{x})=u\mathbf{1_N}
$$
where $u \in \mathbb{R}$ is a scalar, $\mathbf{1_N}\in \mathbb{R}^N$ is the constant vector of 1's, $\mathbf{A}\in \mathbb{R}^{N*N}$ is a scaled tridaigonal matix with 2 on the main diagonal and -1 on the neighboring minor diagonals and the scaling constant is given by $\frac{1}{h^2}$, and $\mathbf{G}:\mathbb{R}^N\to\mathbb{R}^N$ is a vector valued funcition with $x_i^3$ in each component. 

Similarly after discretizing $\lambda$ in time, we can write the other system of equations ivolving $\lambda$ in matrix vector format,
$$
\mathbf{A}\mathbf{\lambda}+\mathbf{F}(\mathbf{x,\lambda})= \mathbf{x} - \mathbf{\overline{x}}
$$
where $\mathbf{F}:\mathbb{R}^N\to\mathbb{R}^N$ is a vector valued funcition with $3x_i^2*\lambda_i$ in each component, and $\mathbf{\overline{x}}$ is a time discretized vector with components based on the given goal function provided.

In [10]:
#Import needed packages

import numpy as np
import scipy
import sympy
from scipy.optimize import fsolve

In [65]:
# Define constants
omega = 1
N=99
h=1/(N+1)

# Define A (discrete Laplace) matrix
main_diag = 2 * np.eye(N)
minor_diags = -1 * (np.eye(N, k=-1) + np.eye(N, k=1))
A = 1/(h*h) * (main_diag + minor_diags)

# Create initial guess vector u for fsolve, and initlial vectors needed to define xbar vector
u_guess = 0
u_guess_vect = u_guess * np.ones(N)
t_nodes = np.zeros(N)
x_bar = np.zeros(N)

# Create xbar vector based off xbar(t) = sin(pi*t)
def xbar(t):
    return np.sin(np.pi * t)

for i in range(N):
    t_nodes[i] = i*h
    x_bar[i] = xbar(t_nodes[i])
    
# Create initial guess vector x for fsolve with 0 boundary conditions
x_guess = np.zeros(N)
x_guess[0] = 0
x_guess[-1] = 0

# Create initial guess vector x for fsolve with 0 boundary conditions
lamb_guess = np.zeros(N)
lamb_guess[0] = 0
lamb_guess[-1] = 0

# Define nonlinear system of equations for x BVP to find roots of using fsolve
def xequation(x, A, u):
    return A @ x + x**3 - u

# Define nonlinear system of equations for lambda BVP to find roots of using fsolve
def lambequation(lamb, A, x, xbar):
    return A @ lamb + 3*(x**2) * lamb - x + xbar

# Use fsolve to find approximate solution of x BVP
resultx = fsolve(xequation, x_guess, args=(A, u_guess_vect), xtol=1e-8)
print("Solution for x:", resultx)

# Use fsolve to find approximate solution of lambda BVP
resultlamb = fsolve(lambequation, lamb_guess, args=(A, resultx, x_bar), xtol=1e-8)
print("Solution for lambda:", resultlamb)

# Define steepest descent move for new u_guess value
def steepest_descent_move(gradient, u_guess, step):
    return u_guess - (step * gradient)

# Define gradient with respect to u of the cost function J
def gradju(omega, u_guess, resultlamb):
    integral = np.trapz(resultlamb, dx=1/len(resultlamb))
    return omega * u_guess + integral

Solution for x: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
Solution for lambda: [-0.00311918 -0.00623836 -0.0093544  -0.01246416 -0.01556451 -0.01865233
 -0.0217245  -0.02477793 -0.02780955 -0.0308163  -0.03379516 -0.03674311
 -0.03965718 -0.04253445 -0.045372   -0.04816697 -0.05091654 -0.05361794
 -0.05626843 -0.05886534 -0.06140604 -0.06388796 -0.0663086  -0.06866549
 -0.07095625 -0.07317855 -0.07533014 -0.07740884 -0.07941253 -0.08133916
 -0.08318678 -0.0849535  -0.08663751 -0.08823708 -0.08975058 -0.09117646
 -0.09251323 -0.09375951 -0.09491403 -0.09597556 -0.09694301 -0.09781535
 -0.09859166 -0.09927111 -0.09985298 -0.10033661 -0.10072147 -0.10100713
 -0.10119322 -0.10127952 -0.10126586 -0.1011522  -0.1009386  -0.10062519
 -0

In [66]:
maxiter = 10000
tol = 1e-8
step = 0.01
iteration = 0

while (iteration < maxiter):
    gradient_mag = np.abs(gradju(omega, u_guess, resultlamb))
    
    if gradient_mag < tol:
        print(f"Converged after {iteration + 1} iterations.")
        print(f"Optimal u value:", u_guess)
        break
    
    else:
        u_guess = steepest_descent_move(gradient, u_guess, step)
        u_guess_vect = u_guess * np.ones(N)
        resultx = fsolve(xequation, x_guess, args=(A, u_guess_vect), xtol=1e-8)
        resultlamb = fsolve(lambequation, lamb_guess, args=(A, resultx, x_bar), xtol=1e-8)
        gradient = gradju(omega, u_guess, resultlamb)
        
        iteration += 1
        
if iteration == maxiter:
    print("Maximum iterations reached without convergence.")

Converged after 1550 iterations.
Optimal u value: 0.06454629140937759


#### Part 3 (Validate Estimate $u^*$)